In [1]:
import tensorflow as tf

import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Dense, Activation, Dropout, Flatten

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

import numpy as np
import matplotlib.pyplot as plt

from keras.applications.inception_v3 import InceptionV3

from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.callbacks import TensorBoard, ModelCheckpoint

#------------------------------
#cpu - gpu configuration
config = tf.ConfigProto( device_count = {'GPU': 0 , 'CPU': 56} ) #max: 1 gpu, 56 cpu
sess = tf.Session(config=config) 
keras.backend.set_session(sess)
#------------------------------


Using TensorFlow backend.


In [2]:
#variables
num_classes = 7 #angry, disgust, fear, happy, sad, surprise, neutral
batch_size = 256
epochs = 100

In [3]:
#read kaggle facial expression recognition challenge dataset (fer2013.csv)
#https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge

with open("/home/topica/workspace/dataset/train.csv") as f:
    content = f.readlines()

lines = np.array(content)

num_of_instances = lines.size
print("number of instances: ",num_of_instances)
print("instance length: ",len(lines[1].split(",")[1].split(" ")))


number of instances:  4179
instance length:  2304


In [4]:
#------------------------------
#initialize trainset and test set
x_train, y_train, x_test, y_test = [], [], [], []

#------------------------------

In [5]:
#transfer train and test set data
for i in range(1,num_of_instances):
    try:
        emotion= lines[i].split(",")[0]
        
        img = lines[i].split(",")[1]
        img = img.strip('\n').replace('\"','')
        val = img.split(" ")
        pixels = np.array(val, 'float32')
        
        emotion = keras.utils.to_categorical(emotion, num_classes)
        y_train.append(emotion)
        x_train.append(pixels)

    except:
        print("",end="")

#------------------------------

In [6]:
#data transformation for train and test sets
x_train = np.array(x_train, 'float32')
y_train = np.array(y_train, 'float32')
x_test = np.array(x_test, 'float32')
y_test = np.array(y_test, 'float32')


x_train_1 = x_train[:4000,:]
y_train_1 = y_train[:4000,:]
x_test = x_train[4000:,:]
y_test = y_train[4000:,:]


x_train /= 255 #normalize inputs between [0, 1]
x_test /= 255

x_train = x_train.reshape(x_train.shape[0], 48, 48, 1)
x_train = x_train.astype('float32')
x_test = x_test.reshape(x_test.shape[0], 48, 48, 1)
x_test = x_test.astype('float32')

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
#------------------------------

4178 train samples
178 test samples


In [7]:
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0,
                          write_graph=True, write_images=False)
filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint, tensorboard ]

In [8]:
# base_model = InceptionV3(weights='imagenet', include_top=False)

# # add a global spatial average pooling layer
# x = base_model.output
# x = GlobalAveragePooling2D()(x)
# # let's add a fully-connected layer
# x = Dense(1024, activation='relu')(x)
# # and a logistic layer -- let's say we have 200 classes
# predictions = Dense(7, activation='softmax')(x)

# # this is the model we will train
# model = Model(inputs=base_model.input, outputs=predictions)


In [9]:
from keras.models import model_from_json

model = model_from_json(open("/home/topica/workspace/Facial-Expression-Recognition/model_4layer_2_2_pool.json", "r").read())
model.load_weights("/home/topica/workspace/Facial-Expression-Recognition/model_4layer_2_2_pool.h5")

In [10]:
len(model.layers)

30

In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 48, 64)        640       
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 64)        256       
_________________________________________________________________
activation_1 (Activation)    (None, 48, 48, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 128)       204928    
_________________________________________________________________
batch_normalization_2 (Batch (None, 24, 24, 128)       512       
__________

In [12]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

NameError: name 'base_model' is not defined

In [13]:
#batch process
gen = ImageDataGenerator()
train_generator = gen.flow(x_train, y_train, batch_size=batch_size)
test_generator = gen.flow(x_test, y_test, batch_size=batch_size)

In [14]:
model.compile(loss='categorical_crossentropy'
    , optimizer=keras.optimizers.Adam()
    , metrics=['accuracy'])

In [ ]:
model.fit_generator(generator=train_generator, steps_per_epoch=batch_size, validation_data=test_generator,validation_steps=batch_size, epochs=epochs, callbacks=callbacks_list )

Epoch 1/100
256/256 [==============================] - 2274s 9s/step - loss: 0.7701 - acc: 0.7730 - val_loss: 1.8675 - val_acc: 0.2865

Epoch 00001: val_acc improved from -inf to 0.28652, saving model to weights-improvement-01-0.29.h5
Epoch 2/100
256/256 [==============================] - 2208s 9s/step - loss: 0.2345 - acc: 0.9149 - val_loss: 1.8517 - val_acc: 0.2865

Epoch 00002: val_acc did not improve from 0.28652
Epoch 3/100
256/256 [==============================] - 2252s 9s/step - loss: 0.1383 - acc: 0.9495 - val_loss: 1.8514 - val_acc: 0.2865

Epoch 00003: val_acc did not improve from 0.28652
Epoch 4/100
256/256 [==============================] - 2326s 9s/step - loss: 0.1135 - acc: 0.9585 - val_loss: 1.8492 - val_acc: 0.2865

Epoch 00004: val_acc did not improve from 0.28652
Epoch 5/100
256/256 [==============================] - 2264s 9s/step - loss: 0.1030 - acc: 0.9615 - val_loss: 1.8485 - val_acc: 0.2865

Epoch 00005: val_acc did not improve from 0.28652
Epoch 6/100
256/256 [

In [ ]:
model.save_weights('weights_30_08.h5')